In [47]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras # keras
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import warnings
import re

import bs4

warnings.filterwarnings("ignore", category=DeprecationWarning)
%matplotlib inline

In [48]:
df = pd.read_csv('tweets8.csv')

In [49]:
df1 = pd.read_csv('tweets1.csv')
df2 = pd.read_csv('tweets2.csv')
df3 = pd.read_csv('tweets3.csv')
df4 = pd.read_csv('tweets4.csv')
df5 = pd.read_csv('tweets5.csv')
df6 = pd.read_csv('tweets6.csv')
df7 = pd.read_csv('tweets7.csv')

In [50]:
df.text

0      RT @MistressDeee: Down bitch where you belong ...
1            RT @ArianaGrande: 🖤 https://t.co/0B21fht9Y5
2      RT @Nuzzies_: Omg what in the world 😭😂 https:/...
3      Taking some inspiration from @needtobreathe ne...
4      RT @Masudaliii: Note how this employee said “w...
5      Who was root of Russian collusion in the 2016 ...
6      RT @HoarseWisperer: Were it not for the outrig...
7      RT @maruti_bitamin: dreaming of the sea https:...
8      RT @Sudeep_Trends: 300k up..! Speed up guys......
9      Weekend Favs November 17\n\nWeekend Favs Novem...
10     RT @singrewind: THE COMPLETE SAGA https://t.co...
11     RT @WyattEarpLA: Our President needs our praye...
12     RT @KarenPBuckMP: ‘From 1997 to 2012 the UK wa...
13     RT @S_for_Shadows: The talented @Sophia_Walker...
14     Visit the Christmas Village and see Semi Preci...
15     RT @destinysarahi: Now this is a fucking threa...
16     RT @shattawalegh: At the end Ghana wins. 🇬🇭 \n...
17     RT @GoggleboxQuotes: So 

In [51]:
#df.created=df.created.fillna(df.date)

In [52]:
#df=df.drop(['date'], axis=1)
#df=df.drop(['Unnamed: 0'], axis=1)

Dataset cleaner and ready to create dictionary

In [53]:
stations = ['23 Dry Dock Avenue','27 Dry Dock Avenue','Dry Dock','88 Black Falcon Avenue','Black Falcon',
            'Airport','Alewife', 'Allston Street','Allston St','Andrew','Aquarium','Arlington','Ashmont','Assembly',
            'Babcock Street','Babcock','Back Bay', 'Back of the Hill','Beachmont','Beaconsfield','Bellingham Square',
            'Bellingham','Blandford Street','Blanford St', 'Boston College','BC','Boston University Central',
            'BU Central','Boston University East','BU East','Boston University West','BU West','BU','Bowdoin',
            'Box District','Boylston','Braintree','Brandon Hall','Brigham Circle','Brigham','Broadway',
            'Brookline Hills','Brookline Village','Butler','Capen Street', 'Capen St','Cedar Grove','Central',
            'Central Avenue','Charles/MGH','Charles','MGH','Chelsea','Chestnut Hill','Chestnut Hill Avenue',
            'Chestnut Hill Ave','Chinatown','Chiswick Road','Chiswick','Cleveland Circle','CC','Community College',
            'Bunker Hill','Coolidge Corner','Coolidge','Copley','Courthouse','Davis','Dean Road','Dean Rd',
            'Design Center','Downtown Crossing','Downtown Xing','Dudley Square','East Berkeley Street',
            'East Berkley','E. Berkley','Eastern Avenue','Eastern Ave','Eliot','Englewood Avenue','Englewood',
            'Englewood Ave', 'Fairbanks','Fairbanks St','Fenway','Fenwood Road', 'Fenwood','Fenwood Rd',
            'Fields Corner','Fields','Forest Hills','Government Center','Green','Griggs Street/Long Avenue',
            'Griggs St','Griggs Street','Long Ave','Harvard', 'Harvard Avenue','Harvard Ave','Harvard St',
            'Hawes Street','Hawes','Hawes St','Haymarket','Heath Street','Heath St', 'Herald Street','Herald St',
            'Hynes Convention Center', 'Hynes','Jackson Square','Jackson Sq','JFK/UMass','JFK','UMass','Kendall/MIT',
            'Kendall','MIT','Kenmore','Kent Street','Kent St','Lechmere','Lenox Street','Lenox St','Longwood',
            'Longwood Medical Area','Malden Center','Massachusetts Ave','Mass Ave','Massachusetts Avenue','Mattapan',
            'Maverick','Melnea Cass Boulevard','Melnea Cass', 'Melnea Cass Blvd','Melania Cass','Milton',
            'Mission Park','Museum of Fine Arts','MFA','Newton Centre','Newton Center','Newton Highlands',
            'Newton Street','Newton','North Quincy','North Station','Northeastern University','Northeastern',
            'Northern Avenue & Harbor Street','Northern Avenue & Tide Street','Oak Grove','Orient Heights',
            'Packards Corner','Packards','Park Street','Park St','Pleasant St','Pleasant Street','Porter',
            'Prudential','Quincy Adams','Quincy Center','Quincy','Reservoir','Revere Beach','Revere','Riverside',
            'Riverway','Roxbury Crossing','Ruggles','Savin Hill','Savin','Science Park','Shawmut','Silver Line Way',
            'South Station','South Street','South St','St. Marys Street','St. Paul Street','St. Paul St','St Marys',
            'State','State St','State Street','Stony Brook','Suffolk Downs','Sullivan Square','Sullivan Sq',
            'Summit Avenue','Summit Ave','Sutherland Road','Sutherland Rd','Symphony', 'Tappan Street','Tappan St',
            'Terminal A','Terminal B','Terminal B Stop 1','Terminal B Stop 2','Terminal C','Terminal E',
            'Tufts Medical Center', 'Tufts', 'Union Park Street', 'Union','Valley Road','Waban', 'Warren Street', 
            'Warren St', 'Washington','Washington Square','Washington Sq','Washington Street', 'Washington St',
            'Wellington','Wollaston', 'Wonderland', 'Wood Island','Woodland','Worcester Square', 'World Trade Center',
           'Green Line', 'Orange Line', 'Red Line', 'Blue Line', 'Commuter Rail', 'Silver Line', 'SL1',
           'SL2','SL3','SL4','Orange','Red','Blue','Silver','Route','Bus','the T', 'B line','C line', 'D line',
           'E line','Fairmount','Fitchburg','Framingham','Worcester','Franklin','Greenbush','Haverhill','Kingston',
           'Plymouth','Lowell','Middleborough','Lakeville','Needham','Newburyport','Rockport','Providence','PVD',
           'Stoughton','Foxboro']

In [54]:
stations

['23 Dry Dock Avenue',
 '27 Dry Dock Avenue',
 'Dry Dock',
 '88 Black Falcon Avenue',
 'Black Falcon',
 'Airport',
 'Alewife',
 'Allston Street',
 'Allston St',
 'Andrew',
 'Aquarium',
 'Arlington',
 'Ashmont',
 'Assembly',
 'Babcock Street',
 'Babcock',
 'Back Bay',
 'Back of the Hill',
 'Beachmont',
 'Beaconsfield',
 'Bellingham Square',
 'Bellingham',
 'Blandford Street',
 'Blanford St',
 'Boston College',
 'BC',
 'Boston University Central',
 'BU Central',
 'Boston University East',
 'BU East',
 'Boston University West',
 'BU West',
 'BU',
 'Bowdoin',
 'Box District',
 'Boylston',
 'Braintree',
 'Brandon Hall',
 'Brigham Circle',
 'Brigham',
 'Broadway',
 'Brookline Hills',
 'Brookline Village',
 'Butler',
 'Capen Street',
 'Capen St',
 'Cedar Grove',
 'Central',
 'Central Avenue',
 'Charles/MGH',
 'Charles',
 'MGH',
 'Chelsea',
 'Chestnut Hill',
 'Chestnut Hill Avenue',
 'Chestnut Hill Ave',
 'Chinatown',
 'Chiswick Road',
 'Chiswick',
 'Cleveland Circle',
 'CC',
 'Community Colle

In [55]:
df.count()

id                  729
id_str              729
created             729
text                729
fav_count           729
user_name           729
user_description    637
user_location       558
user_created        729
user_followers      729
dtype: int64

In [56]:
stations_t=tuple(stations)

In [57]:
df_f = df


In [58]:
df1.columns

Index(['Unnamed: 0', 'date', 'favorite_count', 'text', 'user'], dtype='object')

In [59]:
df.columns

Index(['id', 'id_str', 'created', 'text', 'fav_count', 'user_name',
       'user_description', 'user_location', 'user_created', 'user_followers'],
      dtype='object')

In [60]:
df_f['tweet']=df_f['text'].astype('str')

In [61]:
df_f.tweet[0]

'RT @MistressDeee: Down bitch where you belong licking the ground I walk on ! #Goddess @RT4Dommes @RT4Domme @RT4FD @RTplaything @slavekane @…'

In [62]:
frames = [df, df1, df2, df3, df4, df5, df6, df7]
a=[]
for df_f in frames:
    df_f.drop_duplicates(keep='first')
    for row in df_f.iterrows(): 
        match = [station if station in row[1].text else None for station in stations]
        match = [x for x in match if x is not None]
        if len(match)>0:
            if 'date' in df_f.columns:
                a.append([row[1].date, row[1].user, '', row[1].text, ','.join(match)])
            else:
                a.append([row[1].created, row[1].user_name, row[1].user_location, row[1].text, ','.join(match)])                

In [63]:
a

[['2018-11-17T18:53:03',
  'uiltexas',
  'Austin, TX',
  'CONF 3A Final: Callisburg def. Boyd, 3-1, to win its first #UILState Volleyball Championship. Congratulations to th… https://t.co/WX6HDROpxH',
  'State'],
 ['2018-11-17T18:53:04',
  'BillHai61993017',
  'Abingdon, MD',
  'RT @marklevinshow: Coming up tomorrow night at 10 pm on Fox, my interview with the brilliant Victor Davis Hanson on Life, Liberty &amp; Levin!…',
  'Davis'],
 ['2018-11-17T18:53:04',
  'costellodaniel1',
  'Canada Global Contract Search',
  'RT @RedFoxGraphics6: @moncartoons @Paola_Dec1231 @costellodaniel1 @CallienSolace @michael_house9 Yes the UN still think Global warming is i…',
  'Red'],
 ['2018-11-17T18:53:04',
  'Neasha_m',
  nan,
  'RT @MuskCoFootball: INTERCEPTED BY MUSKEGON!!\n\nBig Reds pick off Zee East pass and take over at their own 33. @Stickem231 on the pick:\n\n@Th…',
  'Red'],
 ['2018-11-17T18:53:04',
  'FoodsJoseph',
  'Drumright, OK',
  'RT @FoodsJoseph: When it’s time to carve the Turkey htt

In [64]:
b = pd.DataFrame(a, columns=['date', 'user','location','tweet', 'match'])

In [65]:
uniquematches = b.groupby('match').nunique()

In [66]:
uniquematches.sort_values(['tweet'], ascending=[False])

,date,user,location,tweet,match
match,,,,,
Route,27,6,3,29,1
Newburyport,8,4,1,10,1
Union,12,3,4,9,1
the T,7,7,5,7,1
Green,8,4,3,7,1
Blue,7,7,3,7,1
"North Station,Fitchburg",6,2,1,6,1
"North Station,Newburyport,Rockport",6,3,1,6,1
"Green,Greenbush",5,3,1,5,1


In [69]:
t5 = uniquematches.sort_values(['tweet'], ascending=[False]).head(5)

In [76]:
t5=t5[['tweet']]

In [77]:
t5

,tweet
match,
Route,29
Newburyport,10
Union,9
the T,7
Green,7
